In [1]:
import os
import torch
from torch.utils.data import random_split
import torchio as tio
from pathlib import Path
import importlib
from utils import validate_segmentation, plot_prediction, dice_score


import torchio_utils
importlib.reload(torchio_utils)
from torchio_utils import torchio_compose_train, plot_example


#check if colab is being used
if os.path.exists('/content'):
    print('Running on CoLab')
    from google.colab import drive
    drive.mount('/content/drive')
    torch.cuda.empty_cache()

dataset_path = 'data.zip'
dataset_dir_name = 'data'
dataset_dir = Path(dataset_dir_name)

if not dataset_dir.is_dir():
    !curl --silent --output {dataset_path} --location {dataset_url} 
    !unzip -qq {dataset_path}
!tree -d {dataset_dir_name}

data
├── controls
│   ├── imgs
│   └── labels
└── patients
    ├── imgs
    └── labels

6 directories


In [2]:
images_dir = dataset_dir / 'patients/imgs'
labels_dir = dataset_dir / 'patients/labels'
controls_dir = dataset_dir / 'controls/imgs'
image_paths = sorted(images_dir.glob('*.png'))
label_paths = sorted(labels_dir.glob('*.png'))
control_paths = sorted(controls_dir.glob('*.png'))

torchio_compose_train(image_paths, label_paths, control_paths, 
                                cropsize=(400,991),
                                save_inverted_imgs=False,
                                hist_standardization=True,
                                train_size = 620,
                                include_controls = True,
                                dataset_ID=2)




Found 182 subjects
Saving 182 subjects to nnUNet_raw/Dataset002...


  1%|          | 1/182 [00:00<00:26,  6.81it/s]

0
1


  1%|          | 2/182 [00:00<00:23,  7.57it/s]

2


...

 81%|████████  | 34/42 [00:21<00:06,  1.19it/s]

215


 83%|████████▎ | 35/42 [00:22<00:04,  1.44it/s]

216
217


 86%|████████▌ | 36/42 [00:22<00:03,  1.75it/s]